# 3. Creating your model

In the previous notebooks, we learned how to inspect our data and get it ready for neural network training. Here, we'll see how you can customize a neural network so that you can train it with your data.

## 3.1 PyTorch modules

In PyTorch, all neural networks and neural network layers inherit from [the `torch.nn.Module` base class](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). The most important method of a PyTorch `Module` is [`forward()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.forward), as it defines how a given input is processed to produce the output. We could, for example, create a `Module` that computes the sum of the sine and cosine of the input:

In [1]:
import torch
from torch.nn import Module

class Sumsincos(Module):
    def forward(self, x):
        return torch.sin(x) + torch.cos(x)

To use this `Module`, we first instantiate it and can then call it with an input.

In [2]:
sumsincos = Sumsincos()

x = torch.tensor([0., torch.pi/2, torch.pi, 3*torch.pi/2, 2*torch.pi])

sumsincos(x)

tensor([ 1.0000,  1.0000, -1.0000, -1.0000,  1.0000])

## 3.2 Choosing a model from the TorchVision library

Of course, a `Module` that computes the sum of sines and cosines is of little use if you want to classify Pokémon. Convolutional Neural Networks (CNNs) have a better chance. To limit the scope of this lab, we won't be constructing a CNN layer by layer. Instead, we will see how we can slightly modify existing CNN architectures to work for our purpose.

The TorchVision library contains plenty of established and state-of-the-art CNN models. Feel free to take a look at [this page](https://pytorch.org/vision/stable/models.html). In this notebook, we'll use [ResNet-18](https://pytorch.org/vision/stable/models/generated/torchvision.models.resnet18.html#torchvision.models.resnet18), the tiniest of the popular [ResNet family](https://pytorch.org/vision/stable/models/resnet.html).

Initializing a ResNet-18 model is as simple as:

In [3]:
from torchvision.models import resnet18

rn18 = resnet18()
rn18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

If we initialize the model without any arguments, the model parameters will be initialized with random values. Typically, it is a better idea to use **pretrained parameters**. Let's use the default pretrained weights. For more information on the available weights, see [here](https://pytorch.org/blog/introducing-torchvision-new-multi-weight-support-api/).

In [4]:
rn18 = resnet18(weights='DEFAULT')

## 3.3 Changing the number of output neurons

The ResNet-18 model provided by TorchVision contains 1000 output neurons, as you can see from the `out_features` attribute of the full-connected layer `fc` at the end of the network.

In [5]:
rn18.fc

Linear(in_features=512, out_features=1000, bias=True)

Indeed, if we pass a random tensor through this model, we get 10000 predictions for each batch dimension:

In [6]:
import torch

batch_size = 5
c, h, w = 3, 224, 224

x = torch.randn((batch_size, c, h, w))
rn18(x).shape

torch.Size([5, 1000])

This means that the architecture can be used for classifying 1000 classes. Our Pokémon dataset contains 149 classes, however. So, we need to replace the current fully-connected layer with one that contains 149 output features (make sure to keep the correct amount of input features):

In [7]:
from torch.nn import Linear

rn18.fc = Linear(in_features=512, out_features=42, bias=True)

Let's try to pass the random tensor through the new network...

In [8]:
rn18(x).shape

torch.Size([5, 42])

That's it! Our model is now ready for Pokémon classification! 🥳